In [ ]:

import json
import pandas as pd

# Load one sample file to see the data
sample_file = 'datasets/im3_projected_data_centers/low_growth/low_growth_50_market_gravity.geojson'
print(f"Loading: {sample_file}")

with open(sample_file, 'r')x as f:
    geojson_data = json.load(f)

# Extract just the properties to a DataFrame for easy viewing
features = []
for feature in geojson_data['features']:
    features.append(feature['properties'])

df = pd.DataFrame(features)
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst 5 rows:")
df.head()

Loading: datasets/im3_projected_data_centers/low_growth/low_growth_50_market_gravity.geojson
Dataset shape: (222, 15)
Columns: ['id', 'growth_scenario', 'market_gravity_weight', 'region', 'total_cost_million_usd', 'campus_size_square_ft', 'data_center_it_power_mw', 'mechanical_cooling_frac', 'water_cooling_frac', 'cooling_energy_demand_mwh', 'cooling_water_demand_mgy', 'cooling_water_consumption_mgy', 'normalized_locational_cost', 'normalized_gravity_score', 'weighted_siting_score']

First 5 rows:


,id,growth_scenario,market_gravity_weight,region,total_cost_million_usd,campus_size_square_ft,data_center_it_power_mw,mechanical_cooling_frac,water_cooling_frac,cooling_energy_demand_mwh,cooling_water_demand_mgy,cooling_water_consumption_mgy,normalized_locational_cost,normalized_gravity_score,weighted_siting_score
0,1_0,low,50,alabama,522.04,1000000,36,0.0,1.0,0.0,43.51968,34.815746,0.000182,0.000000,0.000091
1,1_1,low,50,alabama,501.90,1000000,36,0.0,1.0,0.0,43.51968,34.815746,0.000032,0.001216,0.000624
2,4_0,low,50,arizona,499.47,1000000,36,0.0,1.0,0.0,43.51968,34.815746,0.000000,0.424552,0.212276
3,4_1,low,50,arizona,499.47,1000000,36,0.0,1.0,0.0,43.51968,34.815746,0.000000,0.426482,0.213241
4,4_2,low,50,arizona,499.47,1000000,36,0.0,1.0,0.0,43.51968,34.815746,0.000000,0.426835,0.213418


In [8]:
# Let's compare different growth scenarios to understand what they mean
import os

scenarios = ['low_growth', 'moderate_growth', 'high_growth', 'higher_growth']
scenario_data = {}

for scenario in scenarios:
    file_path = f'datasets/im3_projected_data_centers/{scenario}/{scenario}_50_market_gravity.geojson'
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            data = json.load(f)

        # Convert to DataFrame
        features = [feature['properties'] for feature in data['features']]
        df_scenario = pd.DataFrame(features)
        scenario_data[scenario] = df_scenario

        print(f"=== {scenario.upper().replace('_', ' ')} ===")
        print(f"Number of data centers: {len(df_scenario)}")
        print(f"States covered: {df_scenario['region'].nunique()}")
        print(f"Total capacity: {df_scenario['data_center_it_power_mw'].sum()} MW")
        print()

# The key insight: Growth scenarios = how many data centers get built
print("WHAT GROWTH SCENARIOS MEAN:")
print("- Low Growth: Fewest new data centers (conservative demand)")
print("- Moderate Growth: Medium number of new data centers")
print("- High Growth: Many new data centers (high demand)")
print("- Higher Growth: Most new data centers (very high demand)")
print("\nBasically: higher growth = more data centers needed by 2035")


=== LOW GROWTH ===
Number of data centers: 222
States covered: 29
Total capacity: 7992 MW

=== MODERATE GROWTH ===
Number of data centers: 328
States covered: 31
Total capacity: 11808 MW

=== HIGH GROWTH ===
Number of data centers: 909
States covered: 34
Total capacity: 32724 MW

=== HIGHER GROWTH ===
Number of data centers: 1870
States covered: 37
Total capacity: 67320 MW

WHAT GROWTH SCENARIOS MEAN:
- Low Growth: Fewest new data centers (conservative demand)
- Moderate Growth: Medium number of new data centers
- High Growth: Many new data centers (high demand)
- Higher Growth: Most new data centers (very high demand)

Basically: higher growth = more data centers needed by 2035


In [1]:
import folium
import json

# Load the high growth scenario data
sample_file = 'datasets/im3_projected_data_centers/high_growth/high_growth_50_market_gravity.geojson'
with open(sample_file, 'r') as f:
    geojson_data = json.load(f)

print(f"Creating map with {len(geojson_data['features'])} potential data center locations...")

# Create a map centered on the US
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Add the GeoJSON data to the map
folium.GeoJson(
    geojson_data,
    popup=folium.GeoJsonPopup(
        fields=['region', 'total_cost_million_usd', 'cooling_water_consumption_mgy', 'data_center_it_power_mw'],
        aliases=['State:', 'Cost ($M):', 'Water Use (MGY):', 'IT Power (MW):']
    ),
    tooltip=folium.GeoJsonTooltip(
        fields=['region', 'total_cost_million_usd'],
        aliases=['State:', 'Cost ($M):']
    ),
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.3
    }
).add_to(m)

# Save and display
m.save('data_centers_map.html')
print("Interactive map saved as 'data_centers_map.html'")
print("Open this file in your browser to see the map!")
m

Creating map with 909 potential data center locations...
Interactive map saved as 'data_centers_map.html'
Open this file in your browser to see the map!
